#Computer Vision
##Classifying Fruits and Vegetables Using Image Classification
The given dataset contains images of a variety of fruits and vegetables, offering a rich source for developing and testing image recognition algorithms. The food items are categorized as follows:
###Fruits:
**Banana, Apple, Pear, Grapes, Orange, Kiwi, Watermelon, Pomegranate, Pineapple, Mango**
###Vegetables:
**Cucumber, Carrot, Capsicum, Onion, Potato, Lemon, Tomato, Radish, Beetroot, Cabbage,Lettuce, Spinach, Soybean, Cauliflower, Bell Pepper, Chilly, Pepper, Turnip, Corn, Sweetcorn,Sweet Potato, Paprika, Jalapeño, Ginger, Garlic, Peas, Eggplant**


Given this dataset, your task is to create a machine learning model that can classify the imagesinto two main categories: Fruits and Vegetables.

In [21]:
import kagglehub
import os
import shutil

# Download latest version
path = kagglehub.dataset_download("kritikseth/fruit-and-vegetable-image-recognition")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'fruit-and-vegetable-image-recognition' dataset.
Path to dataset files: /kaggle/input/fruit-and-vegetable-image-recognition


In [23]:
import os
import shutil

# Define the base path of the downloaded dataset
base_path = "/kaggle/input/fruit-and-vegetable-image-recognition"

# Define the source directories (train and test within the downloaded dataset)
train_source_dir = os.path.join(base_path, 'train')
test_source_dir = os.path.join(base_path, 'test')

# Define the target directories for the two categories
fruit_train_dir = './fruits_vegetables/train/Fruits'
vegetable_train_dir = './fruits_vegetables/train/Vegetables'
fruit_test_dir = './fruits_vegetables/test/Fruits'
vegetable_test_dir = './fruits_vegetables/test/Vegetables'

# Create the target directories
os.makedirs(fruit_train_dir, exist_ok=True)
os.makedirs(vegetable_train_dir, exist_ok=True)
os.makedirs(fruit_test_dir, exist_ok=True)
os.makedirs(vegetable_test_dir, exist_ok=True)

# Define the list of fruit and vegetable categories based on the dataset description and observed directory names
fruits = ['Banana', 'Apple', 'Pear', 'Grapes', 'Orange', 'Kiwi', 'Watermelon', 'Pomegranate', 'Pineapple', 'Mango', 'orange', 'apple', 'pear', 'grapes', 'kiwi', 'watermelon', 'pomegranate', 'pineapple', 'mango', 'banana']
vegetables = ['Cucumber', 'Carrot', 'Capsicum', 'Onion', 'Potato', 'Lemon', 'Tomato', 'Radish', 'Beetroot', 'Cabbage', 'Lettuce', 'Spinach', 'Soybean', 'Cauliflower', 'Bell Pepper', 'Chilly', 'Pepper', 'Turnip', 'Corn', 'Sweetcorn', 'Sweet Potato', 'Paprika', 'Jalapeño', 'Ginger', 'Garlic', 'Peas', 'Eggplant', 'cucumber', 'carrot', 'capsicum', 'onion', 'potato', 'lemon', 'tomato', 'radish', 'beetroot', 'cabbage', 'lettuce', 'spinach', 'soybeans', 'cauliflower', 'bell pepper', 'chilli pepper', 'pepper bell', 'turnip', 'corn', 'sweet corn', 'sweet potato', 'paprika', 'jalepeno', 'ginger', 'garlic', 'peas', 'eggplant', 'raddish', 'chilli pepper']


# Function to organize images into the target directories
def organize_images(source_dir, fruit_target_dir, vegetable_target_dir):
    for category in os.listdir(source_dir):
        category_path = os.path.join(source_dir, category)
        if os.path.isdir(category_path):
            if category in fruits:
                target_dir = fruit_target_dir
            elif category in vegetables:
                target_dir = vegetable_target_dir
            else:
                print(f"Warning: Category '{category}' not recognized as fruit or vegetable. Skipping.")
                continue

            # Move images to the target directory
            for image_name in os.listdir(category_path):
                source_image_path = os.path.join(category_path, image_name)
                target_image_path = os.path.join(target_dir, image_name)
                shutil.copy(source_image_path, target_image_path) # Use copy to keep original dataset intact

# Organize training and testing images
print("Organizing training images...")
organize_images(train_source_dir, fruit_train_dir, vegetable_train_dir)

print("Organizing testing images...")
organize_images(test_source_dir, fruit_test_dir, vegetable_test_dir) # Corrected the variable name

print("Image organization complete.")

Organizing training images...
Organizing testing images...
Image organization complete.


In [24]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define paths to the organized training and testing directories
train_dir = './fruits_vegetables/train'
test_dir = './fruits_vegetables/test'

# Define image dimensions and batch size
img_width, img_height = 128, 128
batch_size = 32

# Create an image data generator for training with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # Using a split for potential validation later

# Create an image data generator for testing without data augmentation
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess images from the training directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary', # Use 'binary' for binary classification
    subset='training')

# Load and preprocess images from the validation subset of the training directory
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary', # Use 'binary' for binary classification
    subset='validation')


# Load and preprocess images from the testing directory
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary') # Use 'binary' for binary classification

# Get class labels (should be 'Fruits' and 'Vegetables')
class_labels = list(train_generator.class_indices.keys())
print("Class Labels:", class_labels)

Found 344 images belonging to 2 classes.
Found 85 images belonging to 2 classes.
Found 41 images belonging to 2 classes.
Class Labels: ['Fruits', 'Vegetables']


In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [26]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid') # Use 'sigmoid' activation for binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy', # Use 'binary_crossentropy' loss for binary classification
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,939,329 (49.36 MB)

 Trainable params: 12,939,329 (49.36 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
epochs = 10  # Define the number of training epochs

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator # Use the validation generator for monitoring performance during training
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.5198 - loss: 1.1047 - val_accuracy: 0.5882 - val_loss: 0.6781
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.5893 - loss: 0.6740 - val_accuracy: 0.5882 - val_loss: 0.6529
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.6445 - loss: 0.6482 - val_accuracy: 0.6824 - val_loss: 0.6012
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - accuracy: 0.5901 - loss: 0.6076 - val_accuracy: 0.7059 - val_loss: 0.5459
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.7448 - loss: 0.5566 - val_accuracy: 0.5529 - val_loss: 0.5946
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7163 - loss: 0.5270 - val_accuracy: 0.7294 - val_loss: 0.5273
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.6809 - loss: 0.5201 - val_accuracy: 0.6824 - val_loss: 0.5219
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.8080 - loss: 0.4377 - val_accuracy: 0.6706 - val_loss:

In [28]:
loss, accuracy = model.evaluate(test_generator)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.7703 - loss: 0.3996
Test Loss: 0.3932
Test Accuracy: 0.7805
